In [1]:
import torch
from nlp_fast_unlearning.utils import prepare_dbpedia, build_noisy_dl, ensure_deterministic, DEVICE, BATCH_SIZE
from nlp_fast_unlearning.baseline_model import TextClassificationModel

_, _, _, dbpedia_vocab = prepare_dbpedia(for_baseline_only=True)

vocab_size = dbpedia_vocab.vocab_size

baseline_name = "DBpedia_baseline.pt"
unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
unlearning_model.load_state_dict(torch.load(baseline_name))

<All keys matched successfully>

In [2]:
%%time

(
    retain_samples,
    noisy_data,
    retain_valid_dl,
    forget_valid_dl,
    retain_test_dl,
    forget_test_dl,
    dbpedia_vocab,
) = prepare_dbpedia(
    for_baseline_only=False,
    classes_to_forget=[5,6,7,8,9],
    model=unlearning_model,
    retain_percentage=0.01,
    vocab_class=dbpedia_vocab,
)

Searching for error maximizing noise for class  5
Got loss 195.9832305908203 for tensor([844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844, 844,
        844, 844], device='cuda:0')
Searching for error maximizing noise for class  6
Got loss 213.3338623046875 for tensor([101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101,

In [3]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def validation_step(model, batch):
    labels, text, offsets = batch
    out = model(text,offsets)
    loss = F.cross_entropy(out, labels)   
    acc = accuracy(out, labels)
    return {'Loss': loss.detach(), 'Acc': acc}


def validation_epoch_end(model, outputs):
    batch_losses = [x['Loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   
    batch_accs = [x['Acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      
    return {'Loss': epoch_loss.item(), 'Acc': epoch_acc.item()}

In [4]:
import torch.nn.functional as F

@torch.no_grad()
def evaluate_after_unlearning(model, val_loader):
    model.eval()
    outputs = [validation_step(model, batch) for batch in val_loader]
    return validation_epoch_end(model, outputs)

In [5]:
%%time
from tqdm import tqdm


# lrs = [12]
# clips = [22]
# ratios = [1.5]

# The following search candidates take a few minutes to run through
lrs = [7,8,9,10,11,12,13,14]
clips = [17,18,19,20,21,22,23,24]
ratios = [4,8,10,12,15,18]


best_retain_acc = 0

unlearned_model_name = "DBpedia_fast_unlearned.pt"

for lr in tqdm(lrs):
    for clip in clips:
        for ratio in ratios:
            ensure_deterministic()
            
            noisy_loader = build_noisy_dl(
                retain_samples,
                noisy_data,
                dbpedia_vocab,
                retain_to_forget_ratio=ratio,
            )
            unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
            unlearning_model.load_state_dict(torch.load(baseline_name))

            optimizer = torch.optim.SGD(unlearning_model.parameters(), lr = lr)


            unlearning_model.train(True)
            for epoch in range(1):
                running_loss = 0.0
                running_acc = 0
                num_batches = len(noisy_loader)
                
                for i, data in enumerate(noisy_loader):
                    labels, inputs, offsets = data

                    optimizer.zero_grad()
                    outputs = unlearning_model(inputs,offsets)
                    loss = unlearning_model.criterion(outputs, labels)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(unlearning_model.parameters(), clip)
                    optimizer.step()

                    running_loss += loss.item()
                    out = torch.argmax(outputs.detach(),dim=1)
                    running_acc += (labels==out).sum().item()/labels.size(0)
                print(f"Train loss {epoch+1}: {running_loss/num_batches},Train Acc:{running_acc*100/num_batches}%")
                print("Performance of Standard Forget Model on Forget Class")
                history = [evaluate_after_unlearning(unlearning_model, forget_valid_dl)]
                print("Accuracy: {}".format(history[0]["Acc"]*100))
                print("Loss: {}".format(history[0]["Loss"]))

                print("Performance of Standard Forget Model on Retain Class")
                history = [evaluate_after_unlearning(unlearning_model, retain_valid_dl)]
                print("Accuracy: {}".format(history[0]["Acc"]*100))
                print("Loss: {}".format(history[0]["Loss"]))
                forget_acc = evaluate_after_unlearning(unlearning_model, forget_valid_dl)["Acc"]*100
                if forget_acc == 0.0:
                    retain_acc = evaluate_after_unlearning(unlearning_model, retain_valid_dl)["Acc"]*100
                    if retain_acc > best_retain_acc:
                        best_retain_acc = retain_acc
                        best_lr = lr
                        best_clip = clip
                        best_ratio = ratio
                        torch.save(unlearning_model.state_dict(), unlearned_model_name)

print(
    f"Best hyperparams: LR={best_lr}, grad clip={best_clip}, "
    f"ratio={best_ratio} | Best retain acc: {best_retain_acc}"
)
unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
unlearning_model.load_state_dict(torch.load(unlearned_model_name))

  0%|                                                                             | 0/8 [00:00<?, ?it/s]

Train loss 1: 813.4583576571557,Train Acc:64.78914650537635%
Performance of Standard Forget Model on Forget Class
Accuracy: 20.236817002296448
Loss: 68.75955200195312
Performance of Standard Forget Model on Retain Class
Accuracy: 6.58646821975708
Loss: 23.246158599853516
Train loss 1: 302.0453813726252,Train Acc:71.59689529220779%
Performance of Standard Forget Model on Forget Class
Accuracy: 10.185955464839935
Loss: 31.381567001342773
Performance of Standard Forget Model on Retain Class
Accuracy: 95.0734555721283
Loss: 0.414170503616333
Train loss 1: 93.13356750179636,Train Acc:78.21800595238095%
Performance of Standard Forget Model on Forget Class
Accuracy: 25.031903386116028
Loss: 9.26378345489502
Performance of Standard Forget Model on Retain Class
Accuracy: 86.93691492080688
Loss: 0.5897656083106995
Train loss 1: 62.59503196415148,Train Acc:80.61735341643582%
Performance of Standard Forget Model on Forget Class
Accuracy: 20.509839057922363
Loss: 3.2676475048065186
Performance of S

 12%|████████▌                                                           | 1/8 [02:11<15:20, 131.52s/it]

Train loss 1: 968.9072431748913,Train Acc:66.07022849462365%
Performance of Standard Forget Model on Forget Class
Accuracy: 1.2645110487937927
Loss: 41.328617095947266
Performance of Standard Forget Model on Retain Class
Accuracy: 95.05856037139893
Loss: 0.8412427306175232
Train loss 1: 511.0372900529341,Train Acc:70.49898538961038%
Performance of Standard Forget Model on Forget Class
Accuracy: 2.193460799753666
Loss: 32.08176803588867
Performance of Standard Forget Model on Retain Class
Accuracy: 95.91948986053467
Loss: 0.21027107536792755
Train loss 1: 438.7252793194327,Train Acc:51.58110119047619%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.0
Loss: 132.43077087402344
Performance of Standard Forget Model on Retain Class
Accuracy: 73.72954487800598
Loss: 29.098426818847656
Train loss 1: 84.36116173392848,Train Acc:81.50572772391506%
Performance of Standard Forget Model on Forget Class
Accuracy: 17.742924392223358
Loss: 10.286556243896484
Performance of Standard Fo

 25%|█████████████████                                                   | 2/8 [04:27<13:25, 134.19s/it]

...

 88%|███████████████████████████████████████████████████████████▌        | 7/8 [16:30<02:24, 144.63s/it]

Train loss 1: 3112.6291727865896,Train Acc:48.214885752688176%
Performance of Standard Forget Model on Forget Class
Accuracy: 22.1073716878891
Loss: 210.71705627441406
Performance of Standard Forget Model on Retain Class
Accuracy: 4.82182539999485
Loss: 66.48725891113281
Train loss 1: 1866.4179874766958,Train Acc:44.51887175324675%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.020032051543239504
Loss: 281.2443542480469
Performance of Standard Forget Model on Retain Class
Accuracy: 75.77904462814331
Loss: 6.745492935180664
Train loss 1: 1740.966783796038,Train Acc:33.61235119047619%
Performance of Standard Forget Model on Forget Class
Accuracy: 21.45378887653351
Loss: 83.48030853271484
Performance of Standard Forget Model on Retain Class
Accuracy: 6.418757885694504
Loss: 91.76441192626953
Train loss 1: 1359.4770628276624,Train Acc:30.70716470452447%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.030048078042455018
Loss: 240.08441162109375
Performance

100%|████████████████████████████████████████████████████████████████████| 8/8 [18:52<00:00, 141.53s/it]


Best hyperparams: LR=8, grad clip=19, ratio=10 | Best retain acc: 95.72798013687134
CPU times: user 18min 5s, sys: 45.2 s, total: 18min 50s
Wall time: 18min 52s


<All keys matched successfully>

In [6]:
print("Performance of Standard Forget Model on Forget Class")
history = [evaluate_after_unlearning(unlearning_model, forget_valid_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

print("Performance of Standard Forget Model on Retain Class")
history = [evaluate_after_unlearning(unlearning_model, retain_valid_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

Performance of Standard Forget Model on Forget Class
Accuracy: 0.0
Loss: 132.2335968017578
Performance of Standard Forget Model on Retain Class
Accuracy: 95.72389721870422
Loss: 0.4859219193458557


In [7]:
print("Test on Forget Class")
history = [evaluate_after_unlearning(unlearning_model, forget_test_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

print("Test on Retain Class")
history = [evaluate_after_unlearning(unlearning_model, retain_test_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

Test on Forget Class
Accuracy: 0.0
Loss: 131.6812744140625
Test on Retain Class
Accuracy: 95.88307738304138
Loss: 0.3123016953468323
